In [2]:
# Program 4: Study the effect of batch normalization and dropout in neural network classifier.
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Load MNIST Dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define a Simple Feedforward Neural Network
class SimpleNN(nn.Module):
    def __init__(self, use_bn=False, use_dropout=False):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.bn1 = nn.BatchNorm1d(256) if use_bn else nn.Identity()
        self.dropout1 = nn.Dropout(0.5) if use_dropout else nn.Identity()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten input
        x = self.fc1(x)
        x = self.bn1(x)   # Apply Batch Norm (if enabled)
        x = torch.relu(x)
        x = self.dropout1(x)  # Apply Dropout (if enabled)
        x = self.fc2(x)
        return x

def train_and_evaluate(model, optimizer, criterion, epochs=5):
    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            output = model(images)
            _, predicted = torch.max(output, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

model_basic = SimpleNN()
model_bn = SimpleNN(use_bn=True)
model_dropout = SimpleNN(use_dropout=True)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer_basic = optim.Adam(model_basic.parameters(), lr=0.001)
optimizer_bn = optim.Adam(model_bn.parameters(), lr=0.001)
optimizer_dropout = optim.Adam(model_dropout.parameters(), lr=0.001)

# Train and Evaluate Models
acc_basic = train_and_evaluate(model_basic, optimizer_basic, criterion)
acc_bn = train_and_evaluate(model_bn, optimizer_bn, criterion)
acc_dropout = train_and_evaluate(model_dropout, optimizer_dropout, criterion)


print(f"Accuracy without BN & Dropout: {acc_basic:.2f}%")
print(f"Accuracy with Batch Normalization: {acc_bn:.2f}%")
print(f"Accuracy with Dropout: {acc_dropout:.2f}%")

Accuracy without BN & Dropout: 97.82%
Accuracy with Batch Normalization: 97.70%
Accuracy with Dropout: 97.45%
